In [3]:
using Erdos
using Distributions
using Gadfly
using Clustering
using MatrixNetworks

A0 = readdlm("Data/PolBlogs/PolBlogsAdj.txt");
lab = readdlm("Data/PolBlogs/LabelsPolBlogs.txt");

const dim0 = size(A0);
A  = Array{Int64,2}(dim0[1],dim0[1]);
for i =1:dim0[1]
	for j=1:dim0[1]
		A[i,j] = Int64(A0[i,j]);
	end
end
A = A+A';
I,J,V = findnz(A);	
A0 = sparse(I,J,vec(ones(Int64,length(V),1)),dim0[1],dim0[1]);
A0 = A0+A0';
A,p = largest_component(A0);
lab = lab[p];

A0=0;p=0;
gc();

N = size(A,2);

In [4]:
d = sum(A,2);
s = sum(d);

r = 80;
N = length(d); 

v0 = rand(Uniform(-1,1),N,r);
v0 = v0./sqrt.(sum(abs2, v0, 1));
v0, t = qr(v0);
l0 = diag(t);


### Preprocess ###
u0 = rand(Uniform(-1,1),N,1);
u0 = u0./norm(u0);
n=0;
for it = 1:30
    u0 = A*u0 .- (d/s)*(d'*u0);
    n = norm(u0,2);
    u0 = u0/n;
end
shift = n;

l = l0;
##################

it_max=10000;
it = 1;
while it <=it_max
    v0 = A*v0 .- (d/s)*(d'*v0) .+ shift*v0;
    v0, t = qr(v0);
    l = diag(t);
    if norm(vec(abs.(l))-vec(abs.(l0)))<1e-03
             break;
    end
    l0 = l;
    it = it+1;
end

if it ==it_max
        print("No convergence")
end 

print(l0-shift)

[123.169, 51.0163, 41.3968, 36.8346, 35.9999, 33.0616, 32.5806, 30.1114, 28.5776, 28.4128, 26.3173, 26.6059, 25.1424, 24.982, 24.4085, 24.1939, 24.0028, 23.2689, 23.1323, 22.7834, 22.5032, 22.1997, 21.9762, 21.1141, 21.2593, 21.1947, 20.8238, 20.633, 20.4718, 20.0551, 19.803, 19.6648, 19.7376, 19.4592, 19.2395, 19.1697, 18.991, 18.4568, 18.4213, 18.3335, 18.245, 17.8661, 17.7608, 17.74, 17.4554, 17.2666, 17.1275, 17.1315, 16.9504, 16.7333, 16.6445, 16.451, 16.3628, 16.263, 16.1821, 15.9199, 15.8763, 15.7423, 15.5924, 15.2622, 15.3929, 15.293, 15.1127, 14.7858, 14.6541, 14.5646, 14.4437, 14.3822, 14.0104, 14.218, 14.1028, 13.9057, 13.9804, 13.6388, 13.6978, 13.3083, 13.4134, 13.258, 13.2722, 13.0431]

In [6]:
include("Algo/UpdateCentroids.jl")
include("Algo/FindCommunity.jl")
include("Algo/Reorder.jl")
include("Algo/Partition.jl")

n_clusters = 200;
n_it_Kmeans = 100;
n_updates = 100;
community = vec(zeros(N,1));
community0 = vec(zeros(N,1));

Q_best_best = 0;
n_c_best_best = 0;
n_updates_best_best = 0;
r_best = 2;
for r=2:80
    
    Sig = v0[:,1:r]';
    Q_best = 0;
    n_c_best = 0;
    n_c = 0;
    # Probability mass for sampling the initial centroids
    p = vec(d/s);
    n_updates_best = 0;

    for k =1:n_it_Kmeans

        community0, Q, n_updates = Partition(A,Sig,n_updates,n_clusters,p);
        n_c = length(unique(community0));
            if Q>Q_best
            community = community0;
            Q_best = Q;
            n_c_best = n_c;
            n_updates_best = n_updates;
            end
    end	

    community = Reorder(community);
    
    if Q_best>Q_best_best 
        r_best = r;
        Q_best_best = Q_best;
        n_c_best_best = n_c_best;
        n_updates_best_best = n_updates_best;
    end
    
end


	print("Number of updates: ")
	print(n_updates_best_best )
	print("\n")
	print("Number of communities: ")
	print(n_c_best_best )
	print("\n")

	print("Modularity: ")
	print(Q_best_best )
	print("\n")
	print("n: ")
	print(r_best )
	print("\n")


Number of updates: 7
Number of communities: 3
Modularity: 0.3900213426379174
n: 2
